# Parameter Choosing DE Daten KI

Rodar isso na máquina virtual
Revisar os paths dos arquivos!

In [ ]:
# Importando tudo que já está fixo:

# Data processing
import json
import numpy as np
import pandas as pd
import re


# Topic model
import umap
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from hdbscan import HDBSCAN

# Dimension reduction
from umap import UMAP


# Countvectorizer (para stopwords)
from sklearn.feature_extraction.text import CountVectorizer

## For UMAP Analisys:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
%matplotlib inline
sns.set()


from sentence_transformers import SentenceTransformer

# para montar as combinacoes:
from itertools import product

# Para a matrix 
from sklearn.metrics.pairwise import cosine_similarity

# Topic Coherence:
import gensim.corpora as corpora
from gensim.models.coherencemodel import CoherenceModel

## Functions to clean data:

def replace_char(x):
  return re.sub('[^a-zA-Z\süöäß]', '', x)


def clean_corpus(s):
    s = s.apply(lambda x: x.lower() if isinstance(x, str) else x)     # tolower:
    idx = s.apply(lambda x: isinstance(x, str))                       # Criar um índice booleano para filtrar os valores não-texto
    s = s.drop(idx[~idx].index)                                       # Remover os valores não-texto
    s = s.apply(replace_char)                                         # remover as partes de palavras que nao sao caracteres ou espacos
    idx = s.apply(lambda x: len(x) < 2)                               # Criar um índice booleano para filtrar os elementos com comprimento menor que 2
    s = s.drop(idx[idx].index)                                        # Remover os elementos com comprimento menor que 2
    idx = s.apply(lambda x: x == '')                                  # Criar um índice booleano para filtrar os elementos vazios
    s = s.drop(idx[idx].index)                                        # Remover os elementos vazios
    
    return s

def calculuate_coherence_score(topic_model):  
  #variable
  topic_words = topic_words = [[words for words, _ in topic_model.get_topic(topic) if words!=''] 
               for topic in range(len(set(topics))-1)]
  vectorizer = topic_model.vectorizer_model
  tokenizer = vectorizer.build_tokenizer()
  #dictionary
  tokens = [doc.split() for doc in docs]
  dictionary = corpora.Dictionary(tokens)
  corpus = [dictionary.doc2bow(token) for token in tokens]


  coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
  coherence = coherence_model.get_coherence()

  
  return coherence


# creating parameters combinations

n = [5, 7, 10, 20]  # n_neighbors
d = [0.0, 0.1, 0.25]   # min_dist
c = [10, 15, 20, 25, 30] #min_cluster
s = [c_i/4 for c_i in c]  # min_sample

combinations = list(product(n, d, c, s))

combinations = [comb for comb in combinations if comb[3] == comb[2]/4]

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#################### CÉLULA PARA SER MODIFICADA ##########################

# Abrindo o arquivos
df_original = pd.read_json("C:\\Users\\opc\\Documents\\Python_Scripts\\champions_2.0\\01_gesetze_sammlung\\de_concatenado.json")


# German Bund stopwords que eu alterei:
# Carregue a lista a partir do arquivo
with open("C:\\Users\\opc\\Documents\\Python_Scripts\\champions_2.0\\01_gesetze_sammlung\\german_bund_stopwords.json", "r") as f:
    german_stopwords_alterada = json.load(f) 


# 1. Dados com os quais vamos trabalhar:
#daten KI
# 'nationale_datenstrategie', 'nationale_ki_strategie_addendum', 'nationale_ki_strategie'
df_work = df_original.query("documento == 'nationale_datenstrategie' | documento == 'nationale_ki_strategie_addendum' | documento == 'nationale_ki_strategie' ")

# nome dos dados:
tinicio = 'de_daten_ki_'

titulo_df_avg_similarity = 'df_avg_similarity_coerence_coeficient_' + tinicio + '.csv'    


In [ ]:
# vamos ver primeiro o que o daten_ki traz:

docs = df_work['corpus']
docs = clean_corpus(docs).to_list()

### início do loop: rodando todas as combinacoes de parâmetros para fazer o meu modelo:

df = pd.DataFrame({'parameters': [],'n_neighbors': [], 'min_dist': [], 'min_cluster' : [], 
                      'min_sample': [], 'topic_coherence': [], 'nr_topics': [], 'avg_similarity' : [], 'topics': []})

for comb in combinations:
    n_neighbors = comb[0]
    min_dist = comb[1]
    min_cluster = comb[2]
    min_sample = int(comb[3])



    umap_model  = UMAP(n_neighbors=n_neighbors, 
                            min_dist = min_dist,
                            n_components=5, 
                            metric='cosine', random_state=42)


    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster, 
                            min_samples = min_sample, 
                            metric='euclidean', 
                            prediction_data=True)

    vectorizer_model = CountVectorizer(stop_words=german_stopwords_alterada) #german_stop_words alterada

    topic_model = BERTopic(language = "german", vectorizer_model=vectorizer_model, 
                        umap_model=umap_model, 
                        hdbscan_model=hdbscan_model).fit(docs)

    topics, probs = topic_model.fit_transform(docs)

    # Matrix de similaridade:

    matrixA = cosine_similarity(np.array(topic_model.topic_embeddings_)[1:, :])
    avg_similarity = np.average(matrixA)
    labels = (topic_model.get_topic_info().sort_values("Topic", ascending=True).Name)[1:]
    nr_topics = len(labels)
    labels = labels.str.cat(sep='; ')
    title = tinicio + 'n_neighbors = ' + str(n_neighbors) + ' min_dist = ' + str(min_dist) + ' min_cluster = ' + str(min_cluster) + ' min_sample = ' + str(min_sample)
    
    #coherence score:
    cs = calculuate_coherence_score(topic_model)

    df_temp = pd.DataFrame({'parameters': [title],
                            'n_neighbors': [n_neighbors],
                            'min_dist': [min_dist],
                            'min_cluster' : [min_cluster],
                            'min_sample': [min_sample], 
                            'topic_coherence': [cs],
                            'nr_topics': [nr_topics],
                            'avg_similarity' : [avg_similarity],
                            'topics': [labels]})
    

    pd.DataFrame({'parameters': [], 
                       'nr_topics': [], 'avg_similarity' : [], 'topics': []})


    df = pd.concat([df, df_temp])

df.to_csv(titulo_df_avg_similarity, sep = ';', index = False, )

## 1.1 Olhando os dados:

In [ ]:
# reabrindo o arquivo, caso eu tenha fechado
#df = pd.read_csv(titulo_df_avg_similarity, sep=';')

df1 = df
df1 = df1.sort_values('avg_similarity')
df1.head()

In [ ]:
# Verificando a dispersao:

import seaborn as sns
import matplotlib.pyplot as plt

# criando o gráfico de dispersão
sns.scatterplot(data=df1, x='avg_similarity', y='topic_coherence', hue='nr_topics')

# adicionando labels de eixo
plt.xlabel('avg_similarity')
plt.ylabel('topic_coherence')

# adicionando título
titulo_do_plot = tinicio + ": topic coherence X avg similarity"
plt.title(titulo_do_plot)

# exibindo o gráfico
plt.show()

In [ ]:
# !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# Modificar aqui também!

# reabrindo o arquivo, caso eu tenha fechado
df1 = pd.read_csv(titulo_df_avg_similarity, sep=';')

df1 = df1.query('topic_coherence > 0.38 & avg_similarity < 0.775')
df1 = df1.reset_index()

combinations = []

for i in range(0,len(df1)):
    n = df1.iloc[i, 2]  # linha coluna
    d = df1.iloc[i, 3]
    c = int(df1.iloc[i, 4])
    s = int(df1.iloc[i, 5])
    
    com = [(n, d, c, s)]

    combinations.extend(com)

for item in combinations:
    for element in item:
        if type(element) == np.float64:
            element = float(element)

for comb in combinations:
    print(comb)

In [ ]:
combinations

In [ ]:
# Dados

docs = df_work['corpus']
docs = clean_corpus(docs).to_list()

### início do loop:

df2 = pd.DataFrame({'Document': [],'Topic': [], 'Name': [], 'Top_n_words' : [], 'Probability': [], 'Representative_document': [], 'model': []})

for comb in combinations:
    print(comb)


    n_neighbors = int(comb[0])
    min_dist = float(comb[1])
    min_cluster = int(comb[2])
    min_sample = int(comb[3])


    umap_model  = UMAP(n_neighbors=n_neighbors, 
                            min_dist = min_dist,
                            n_components=5, 
                            metric='cosine', random_state=42)


    hdbscan_model = HDBSCAN(min_cluster_size=min_cluster, 
                            min_samples = min_sample, 
                            metric='euclidean', 
                            prediction_data=True)

    vectorizer_model = CountVectorizer(stop_words=german_stopwords_alterada) #german_stop_words alterada

    topic_model = BERTopic(language = "german", vectorizer_model=vectorizer_model, umap_model=umap_model, hdbscan_model=hdbscan_model).fit(docs)

    topics, probs = topic_model.fit_transform(docs)

    # Dataframe with representative examples:

    df_temp = topic_model.get_document_info(docs)
    df_temp = df_temp.query('Representative_document == True')

    #confirmando o modelo desse df
    position = combinations.index(comb)
    position = str(position)

    #add modelo
    df_temp['model'] =  str(comb)

    df2 = pd.concat([df2, df_temp], axis=0)

        ##### Visualizacao dos resultados

    title = tinicio + '_n_neighbors = ' + str(n_neighbors) + ' min_dist = ' + str(min_dist) + ' min_cluster = ' + str(min_cluster) + ' min_sample = ' + str(min_sample)

    fig_1 = topic_model.visualize_heatmap()
    fig_1.update_layout(title = title)

    ############################################Visualizacao da distribuicao

    sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")   #Suitable Score Functions: cosine
    embeddings = sentence_model.encode(docs, show_progress_bar=False)


    # Train BERTopic
    topic_model = BERTopic(language = "german", 
                            umap_model=umap_model,
                            vectorizer_model=vectorizer_model,
                            hdbscan_model=hdbscan_model).fit(docs, embeddings)


    # Reduce dimensionality of embeddings, this step is optional but much faster to perform iteratively:
    reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine', random_state=42).fit_transform(embeddings)
    fig_2 = topic_model.visualize_documents(docs, reduced_embeddings=reduced_embeddings)
    fig_2.update_layout(title = title)

    title = re.sub(r'\s', '_', title) + '.html'

    with open(title, 'a') as f:
        f.write(fig_1.to_html(full_html=False, include_plotlyjs='cdn'))
        f.write(fig_2.to_html(full_html=False, include_plotlyjs='cdn'))

#Fora do loop:

tituloex_models = 'df_examples_models_' + tinicio + '.csv'
df2.to_csv(tituloex_models, sep = ';', index = False, )